In [1]:
!pip install transformers SentencePiece
from huggingface_hub import notebook_login
notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import re
import math
from transformers import XLNetTokenizer, XLNetModel
from google.colab import drive
drive.mount('/content/gdrive')

# Run locally
#DATA_PATH = ''

# KW
DATA_PATH = '/content/gdrive/MyDrive/CSI6900/'

# FZ
#DATA_PATH = 'gdrive/MyDrive/CSI6900/'

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/gdrive


device(type='cuda', index=0)

## Load data

In [3]:
records = pd.read_json(DATA_PATH + 'data/records.json')
records = records.drop(['created_at', 'updated_at', 'deleted_at', 'publish', 'academic_credentials', 'age_max', 'age_min', 'last_name', 
                        'latitude', 'longitude', 'name_of_private_practice', 'fee_description',	'fee_type',	'first_name',	'languages',
                        'organization_type', 'original_id',	'record_type',	'salutation_type', 'website'], axis=1)

taxonomy = pd.read_json(DATA_PATH + 'data/taxonomy_headings.json')
taxonomy = taxonomy.drop(['created_at',	'updated_at',	'deleted_at', 'alias_of_id', 'short_description',	'original_id'], axis=1)

taxonomy.head(10)

,id,name,description,translations
0,1,Root,Root,"{""name"":{""en"":""Root"",""fr"":null},""description"":..."
1,2,All Mental Health Resources,<p>\r\n\tThe listings of mental health resourc...,"{""name"":{""en"":""All Mental Health Resources"",""f..."
2,3,Crisis and Emergency,<p>\r\n\tRefers to all programs that provide i...,"{""name"":{""en"":""Crisis and Emergency"",""fr"":""Res..."
3,4,"System Navigation, including Information and R...","<p>\r\n\tAre you looking for help, but don&#39...","{""name"":{""en"":""System Navigation, including In..."
4,5,Child Welfare including Children's Aid Society...,<p>The child welfare / child protection system...,"{""name"":{""en"":""Child Welfare including Childre..."
5,6,Emergency Shelter and Housing,<p>\r\n\tThere are various shelters that peopl...,"{""name"":{""en"":""Emergency Shelter and Housing"",..."
6,7,Hospital Emergency Department,<p>\r\n\tIs there an emergency such as medical...,"{""name"":{""en"":""Hospital Emergency Department"",..."
7,8,"Crisis Lines including Telephone, Online and Chat",<p>\r\n\tAre you in a crisis? Crisis lines off...,"{""name"":{""en"":""Crisis Lines including Telephon..."
8,9,Psychiatrists,<p>\r\n\tPsychiatrists are medical doctors who...,"{""name"":{""en"":""Psychiatrists"",""fr"":""Psychiatre..."
9,10,A-Z Mental Health Conditions and Topics,<p>\r\n\tAlphabetical list of mental health to...,"{""name"":{""en"":""A-Z Mental Health Conditions an..."


## Pre-processing

In [4]:
!cp /content/gdrive/MyDrive/CSI6900/pre_processing.py /content

In [5]:
from pre_processing import PreProcessing

records, taxonomy = PreProcessing(records, taxonomy).preprocess()

Length of records before preprocessing: 6406
Length of taxonomy before preprocessing: 277
Length of records after preprocessing: 6239
Length of taxonomy after preprocessing: 192


## Generate embeddings

In [6]:
tokenizer = XLNetTokenizer.from_pretrained('AIMH/mental-xlnet-base-cased')
model = XLNetModel.from_pretrained('AIMH/mental-xlnet-base-cased').to(device)
model.eval()

Some weights of the model checkpoint at AIMH/mental-xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLNetModel(
  (word_embedding): Embedding(32000, 768)
  (layer): ModuleList(
    (0-11): 12 x XLNetLayer(
      (rel_attn): XLNetRelativeAttention(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): XLNetFeedForward(
        (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_1): Linear(in_features=768, out_features=3072, bias=True)
        (layer_2): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (activation_function): GELUActivation()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
)

In [7]:
records_description = records['description'].tolist()
tokenized_record = tokenizer(records_description, padding='max_length', max_length=512, truncation=True, return_tensors='pt').to(device)
taxonomy_description = taxonomy['description'].tolist()
tokenized_taxonomy = tokenizer(taxonomy_description, padding='max_length', max_length=512, truncation=True, return_tensors='pt').to(device)

In [8]:
batch_size = 64

num_batches = math.ceil(len(tokenized_record.input_ids)/batch_size)

# Generate embeddings for the tokenized_record using the BERT model in batches
records_embeddings = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    end_idx = end_idx if end_idx <= len(tokenized_record.input_ids) else len(tokenized_record.input_ids)

    batch_input_ids = tokenized_record['input_ids'][start_idx:end_idx].to(device)
    batch_attention_mask = tokenized_record['attention_mask'][start_idx:end_idx].to(device)
    batch_token_type_ids = tokenized_record['token_type_ids'][start_idx:end_idx].to(device)

    with torch.no_grad():
        embedding = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        token_type_ids=batch_token_type_ids)
        embedding = embedding.last_hidden_state.mean(dim=1).cpu()
    
    records_embeddings.append(embedding)

In [9]:
batch_size = 64

num_batches = math.ceil(len(tokenized_taxonomy.input_ids)/batch_size)

# Generate embeddings for the tokenized_taxonomy using the BERT model in batches
taxonomy_embeddings = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    end_idx = end_idx if end_idx <= len(tokenized_taxonomy.input_ids) else len(tokenized_taxonomy.input_ids)

    batch_input_ids = tokenized_taxonomy['input_ids'][start_idx:end_idx].to(device)
    batch_attention_mask = tokenized_taxonomy['attention_mask'][start_idx:end_idx].to(device)
    batch_token_type_ids = tokenized_record['token_type_ids'][start_idx:end_idx].to(device)

    with torch.no_grad():
        embedding = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        token_type_ids=batch_token_type_ids)
        embedding = embedding.last_hidden_state.mean(dim=1).cpu()
    
    taxonomy_embeddings.append(embedding)

## Save embeddings

In [10]:
records_embeddings = torch.cat(records_embeddings, dim=0)
print(records_embeddings.shape)
torch.save(records_embeddings, DATA_PATH + 'data/embeddings/xlnet_records_embeddings.pt')

torch.Size([6239, 768])


In [11]:
taxonomy_embeddings = torch.cat(taxonomy_embeddings, dim=0)
print(taxonomy_embeddings.shape)
torch.save(taxonomy_embeddings, DATA_PATH + 'data/embeddings/xlnet_taxonomy_embeddings.pt')

torch.Size([192, 768])
